In [129]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import time
import os
import pickle

from plotly.subplots import make_subplots
from my_lib.utility import *

## Actividad 3: Algoritmo genético para resolver el problema del viajero frecuente

El problema es de tipo TSP con 18 ciudades del conteninte Americano. Por lo tanto existen $18!\$ soluciones posibles ($6402373705728000$), dada el basto espacio de busqueda la población de individuos se debe limitar a minimo 100 y máximo 500.

> Criterios para la evaluación
> - Implementar 2 métodos de cruza, PMX y algún otro método como OX o CX
> - Implementar 2 métodos de mutación, Scramble y Heuristic
> - Implementar un método de selección, cualquiera de los ya vistos
> - Implementar un método hibrido de paro
> - Implementar un méotodo elitista
> - Realizar las comparaciones y la discusión de los resultados

# Codificación del dominio

In [130]:
ciudades18 = {
    'Ciudad': ['Mexico City', 'Merida', 'Buenos Aires', 'Quito', 'Washington D.C.', 
               'New York', 'Miami', 'Monterrey', 'Panama City', 'San Salvador',
               'Managua', 'Brazilia', 'Mendoza', 'Caracas', 'Montevideo', 
               'Guadalajara', 'Boston', 'Bogotá'],
    'Latitud': [19.4326, 20.9674, -34.6037, -0.1807, 38.9072, 
                40.7128, 25.7617, 25.6866, 8.9824, 13.6929, 
                12.1140, -15.8267, -32.8895, 10.4806, -34.9011, 
                20.6597, 42.3601, 4.7110],
    'Longitud': [-99.1332, -89.5926, -58.3816, -78.4678, -77.0369, 
                 -74.0060, -80.1918, -100.3161, -79.5199, -89.2182, 
                 -86.2362, -47.9218, -68.8458, -66.9036, -56.1645, 
                 -103.3496, -71.0589, -74.0721]
} 
nombre_ciudades = np.array(ciudades18['Ciudad'])

In [131]:
id_ciudades = np.arange(nombre_ciudades.shape[0])
bin_ciudades = binarizar(datos=id_ciudades,nbites=3)

ciudades = np.zeros((nombre_ciudades.shape[0],4),dtype="object")
ciudades[:,0] = nombre_ciudades
ciudades[:,1] = id_ciudades
ciudades[:,2] = bin_ciudades
ciudades[:,3] = np.zeros((nombre_ciudades.shape[0]))
print(ciudades)

[['Mexico City' 0 '000' 0.0]
 ['Merida' 1 '001' 0.0]
 ['Buenos Aires' 2 '010' 0.0]
 ['Quito' 3 '011' 0.0]
 ['Washington D.C.' 4 '100' 0.0]
 ['New York' 5 '101' 0.0]
 ['Miami' 6 '110' 0.0]
 ['Monterrey' 7 '111' 0.0]
 ['Panama City' 8 '1000' 0.0]
 ['San Salvador' 9 '1001' 0.0]
 ['Managua' 10 '1010' 0.0]
 ['Brazilia' 11 '1011' 0.0]
 ['Mendoza' 12 '1100' 0.0]
 ['Caracas' 13 '1101' 0.0]
 ['Montevideo' 14 '1110' 0.0]
 ['Guadalajara' 15 '1111' 0.0]
 ['Boston' 16 '10000' 0.0]
 ['Bogotá' 17 '10001' 0.0]]


## Matriz de distancias

#### Fórmula de Haversine 

$d = 2r \cdot arsin(\sqrt{sin^2(\frac{\Delta\phi}{2}) + cos(\phi_1) \cdot cos(\phi_2) \cdot sin^2(\frac{\Delta\lambda}{2})})$

* d es la distancia entre los dos puntos 
* r = radio de la tierra (6371)
* $\phi_1, \phi_2$ = latitudes de los puntos en rad
* $\Delta\phi$ = diferencia de las latitudes
* $\Delta\lambda$ = diferencia de las longitudes


In [132]:
# Latitud y longitud de cada ciudad

n_ciudades = ciudades.shape[0]
distancias = np.zeros((n_ciudades, n_ciudades))

for i in range(n_ciudades):
    c1_latitud = ciudades18["Latitud"][i]
    c1_longitud = ciudades18["Longitud"][i]
    for j in range(n_ciudades):
        if i != j:
            c2_latitud = ciudades18["Latitud"][j]
            c2_longitud = ciudades18["Longitud"][j]
            distancias[i][j] = get_distancia(
                lat1=c1_latitud, long1=c1_longitud, lat2=c2_latitud, long2=c2_longitud
            )

distancias_df = pd.DataFrame(distancias,columns=ciudades18["Ciudad"],index=ciudades18["Ciudad"])
display(distancias_df)

,Mexico City,Merida,Buenos Aires,Quito,Washington D.C.,New York,Miami,Monterrey,Panama City,San Salvador,Managua,Brazilia,Mendoza,Caracas,Montevideo,Guadalajara,Boston,Bogotá
Mexico City,0.000000,1009.959843,7392.178684,3135.709015,3031.811604,3359.383041,2065.272960,705.925938,2408.857647,1233.956557,1600.957943,6835.065741,6648.609671,3595.449156,7554.599238,461.067374,3665.482788,3170.731318
Merida,1009.959843,0.000000,7004.211339,2644.150812,2327.565260,2643.050506,1097.212425,1213.520720,1715.030837,809.861482,1047.292813,6116.463263,6381.173335,2689.070020,7144.536815,1429.853511,2941.198434,2464.943628
Buenos Aires,7392.178684,7004.211339,0.000000,4360.060643,8396.527908,8526.194114,7094.952486,8028.590573,5333.037977,6280.332978,5968.507501,2334.403612,985.614665,5093.044534,205.232076,7766.608993,8654.780454,4670.520317
Quito,3135.709015,2644.150812,4360.060643,0.000000,4348.827501,4569.460001,2890.594533,3711.248568,1025.530802,1944.679440,1613.735904,3776.161246,3774.047997,1743.741681,4501.636863,3562.151769,4788.167318,730.923189
Washington D.C.,3031.811604,2327.565260,8396.527908,4348.827501,0.000000,327.582714,1491.176569,2623.500270,3336.690529,3048.362836,3114.733025,6797.691847,8028.085133,3317.116974,8483.304262,3229.236038,633.671486,3814.195109
New York,3359.383041,2643.050506,8526.194114,4569.460001,327.582714,0.000000,1757.961225,2945.076096,3570.004517,3348.328915,3397.166346,6839.011011,8201.327390,3433.343519,8603.779701,3554.283691,306.108494,4003.222959
Miami,2065.272960,1097.212425,7094.952486,2890.594533,1491.176569,1757.961225,0.000000,2013.994655,1867.122418,1639.742840,1644.533412,5794.758090,6631.402300,2200.658515,7204.245436,2430.007251,2025.336834,2429.837957
Monterrey,705.925938,1213.520720,8028.590573,3711.248568,2623.500270,2945.076096,2013.994655,0.000000,2876.282439,1766.700637,2111.060860,7310.080266,7315.684383,3898.681265,8184.245928,639.142332,3245.755067,3640.297547
Panama City,2408.857647,1715.030837,5333.037977,1025.530802,3336.690529,3570.004517,1867.122418,2876.282439,0.000000,1179.637483,812.486225,4440.058794,4791.336430,1392.559785,5462.750288,2865.971802,3802.437689,766.254430
San Salvador,1233.956557,809.861482,6280.332978,1944.679440,3048.362836,3348.328915,1639.742840,1766.700637,1179.637483,0.000000,367.803223,5601.769099,5613.686901,2451.382655,6430.418383,1687.991760,3634.269046,1937.742276


# Generación de población

Para poder realizar una buena comparación con las distintas combinaciones de operadores genéticos se trabajará con 5 poblaciones iniciales, las cuales son las mismas parar cada experimento


Si ya existe el archivo de poblaciones no hay necesidad de crear nuevamente las poblaciones iniciales

In [133]:
POB_FILE = "files\poblaciones_3.pkl"

In [134]:
def get_poblaciones_iniciales(n=5, tam_pob=60):
    """
    Genera y carga las poblaciones iniciales para un algoritmo genético.

    Si el archivo de poblaciones no existe, la función lo crea generando las poblaciones iniciales y guardándolas en el archivo.
    Si el archivo ya existe, la función lo carga y devuelve las poblaciones.

    Parámetros:
        n (int): Número de poblaciones iniciales a generar (por defecto, 5).
        tam_pob (int): Tamaño de cada población (por defecto, 60).

    Valor de retorno:
        list: Lista de poblaciones iniciales, donde cada población es una matriz de numpy con 4 columnas: ruta, aptitud, individuo y aptitud.
    """
    poblaciones = []
    POB_PATH = os.path.join("..", POB_FILE)
    if not os.path.exists(POB_PATH):
        print("El archivo de poblaciones no existse, creando archivo")
        
        for i in range(n):
            recorrido = generar_poblacion_perm(ciudades, tam_pob)
            
            poblacion = np.zeros((recorrido.shape[0], 4), dtype="object")
            poblacion[:, 0] = [c.astype(int) for c in recorrido]  # Ruta
            poblacion[:, 1] = np.ones((recorrido.shape[0]))
            poblacion[:,2] = [" ".join([chr(c+64) for c in perm.astype(int)]) for perm in recorrido]
            # poblacion[:, 2] = [individuo_toString(c.astype(int)) for c in recorrido]
            poblacion[:, 3] = np.zeros((recorrido.shape[0]))  # Aptitud
        
            poblaciones.append(poblacion)
        
        try:
            with open(POB_PATH, "wb") as f:
                pickle.dump(poblaciones,f)
            print("Archivo de poblaciones creado correctamente")
        except Exception as e:
            print(f"Error al guardar archivo: {e}")
            
    else:
        try:
            with open(POB_PATH, "rb") as f:
                poblaciones = pickle.load(f)
            print("Archivo de poblaciones cargado correctamente")
        except Exception as e:
            print(f"Error al cargar archivo: {e}")
    
    return poblaciones

In [135]:
pobs = get_poblaciones_iniciales(n=5,tam_pob=100)
print(len(pobs))
print(pobs[0][:2])

Archivo de poblaciones cargado correctamente
5
[[array([12, 16, 13,  3,  9, 14,  7,  1, 15,  0, 10,  6,  4,  5,  8, 17, 11,
          2])
  1.0 'L P M C I N G A O @ J F D E H Q K B' 0.0]
 [array([12, 17,  3,  0, 13,  9, 14, 15,  1,  2, 16,  5,  7,  8, 10,  6,  4,
         11])
  1.0 'L Q C @ M I N O A B P E G H J F D K' 0.0]]


# Función objetivo

In [136]:
def y(ciudad1,ciudad2):
    return distancias[ciudad1,ciudad2]

def evaluar(poblacion):
    evaluacion = np.zeros((poblacion.shape[0]))
    for idx,fila in enumerate(poblacion):
        individuo = fila[0]
        n = len(individuo) - 1
        aptitud = 0
        
        for i in range (n,-1,-1):
            c1 = int(individuo[i]) #Id de la ciudad
            c2 = int(individuo[i-1])
            distancia = y(c1,c2)
            aptitud += distancia
           # print(f"Distancia entre {ciudades[c1]} y {ciudades[c2]}: {distancia}")
        evaluacion[idx] = np.around(aptitud,4)
        
    return evaluacion



In [137]:
def historial_cruces(descendencia):
    hijos = [ind[0] for ind in descendencia]
    padres_codificados = [ind[1] for ind in descendencia]
    padres_codificados = [
        [p.replace(" ", "") for p in padres] for padres in padres_codificados
    ]
    padres_decodificados = [
        [[ord(c) - 64 for c in p] for p in padres] for padres in padres_codificados
    ]
    
    historial = []
    
    for j in range(0,len(hijos),2):
        fila = np.array([padres_decodificados[j][0],padres_decodificados[j][1],hijos[j],hijos[j+1]])
        # print(fila.shape)
        #
        historial.append(fila)
        # print(
        #     f"p1: {padres_decodificados[j][0]} \th1: {hijos[j]}\np2: {padres_decodificados[j][1]} \th2: {hijos[j+1]}"
        # )
        # print(fila)
        # print("")

    return np.array(historial)
    # for j in range(0, len(hijos) - 1):
    #     print(
    #         f"p1: {padres_decodificados[j][0]} \th1: {hijos[j]}\np2: {padres_decodificados[j][1]} \th2: {hijos[j+1]}"
    #     )
    #     print("")

In [138]:
def get_report(n_poblacion,poblacion,time=None):
    info = np.zeros((8))
    info[0] = n_poblacion
    info[1] = np.around(np.mean(poblacion[:,3]),decimals=2)
    info[2] = np.around(np.var(poblacion[:,3]),decimals=2)
    info[3] = np.around(np.std(poblacion[:,3]),decimals=2)
    info[4] = np.around(np.min(poblacion[:,3]),decimals=2)
    info[5] = np.around(np.max(poblacion[:,3]),decimals=2)
    info[6] = np.around(diversity_rate(poblacion),decimals=2)
    if time:
        info[7] = time
    
    return info

In [139]:
def seleccion_cruza(poblacion, op_seleccion=0, op_cruza=3, mode="min"):
    desc = get_next_gen(poblacion, op_seleccion, op_cruza, mode)
    cruces = historial_cruces(desc)
    hijos = np.array([hijo[0] for hijo in desc])
    new_age = poblacion.copy()

    for hijo in hijos:
        nuevo_indiviudo = np.zeros((1, 4), dtype="object")
        nuevo_indiviudo[0, 0] = hijo
        nuevo_indiviudo[0, 1] = 1.0
        nuevo_indiviudo[0, 2] = " ".join([chr(c + 64) for c in hijo])
        nuevo_indiviudo[0, 3] = evaluar(nuevo_indiviudo)[0]
        new_age = np.vstack([new_age, nuevo_indiviudo])
    return new_age,cruces


def msg_info_exp(msg):
    print("===================================================================")
    print(f"{msg}")


def msg_info_inicial(num_pob, poblacion):
    print("===================================================================")
    print(f"Población incial #{num_pob+1}")
    print(f"Costo promedio inicial {np.mean(poblacion[:,3])}")


def msg_info_generacion(num_gen, poblacion, tiempo):
    print("...................................................................")
    print(f"Generación: {num_gen}")
    print(f"Costo promedio: {np.mean(poblacion[:,3])}")
    print(f"Varianza del costo en la genearción: {np.var(poblacion[:,3])}")
    print(f"Desviación estándar del costo en la genearción: {np.std(poblacion[:,3])}")
    print(f"Diversidad promedio: {diversity_rate(poblacion)} genes")
    print(f"Menor costo de toda la generación : {np.min(poblacion[:,3])}")
    print(f"Mayor costo de toda la generación : {np.max(poblacion[:,3])}")
    print(f"Tiempo de ejecución de la genaración {num_gen}: {tiempo}")

    print("...................................................................")


def msg_tiempo_pob(num_pob, tiempo):
    print("--------------------------------------------------------------------")
    print(f"Tiempo de ejecución para la población #{num_pob+1}: {tiempo}")
    print("===================================================================")


def experimentacion(
    poblaciones_iniciales, max_gen=10, op_seleccion=0, op_cruza=3, mode="min",
    mutar = False
):
    start_exp_time = time.time()
    msg_info_exp("Inicio del algoritimo genético")
    gen_results = []

    for i in range(len(poblaciones_iniciales)):
        poblacion = poblaciones_iniciales[i]
        poblacion[:, 3] = evaluar(poblacion)
        poblacion = ordenar_poblacion(poblacion, reverse=True)

        msg_info_inicial(i, poblacion)
        pob_time = 0
        
        historial_generacional = np.zeros((max_gen,2),dtype="object")
        
        for gen in range(max_gen):
            start_gen_time = time.time()
            # if GEN >= MAX_GEN or paro_epsilon(poblacion, 14000, 0.9, opt=1):
            #     print("Convergencia alcanzada")
            #     break

            new_age,cruces = seleccion_cruza(poblacion, 0, 3, "min")
            
            if mutar :
                print("mutar")
                pob_mutante = mutation(new_age,opt=1,operator=0,funcion_aptitud=evaluar,cromosomas_mutation=6)
                poblacion = ordenar_poblacion(pob_mutante,reverse=True)[:poblacion.shape[0]]
            else:
                new_age = ordenar_poblacion(new_age,reverse=True)[:poblacion.shape[0]]
                poblacion = new_age
                
            # pob_mutante = mutation(new_age,opt=1,operator=0,funcion_aptitud=evaluar,cromosomas_mutation=5)
            # poblacion = ordenar_poblacion(pob_mutante,reverse=[True])[:poblacion.shape[0]]
            
            # new_age = ordenar_poblacion(new_age,reverse=True)
            # poblacion = new_age[: poblacion.shape[0]]
            
            
            end_gen_time = time.time()
            elapsed_gen_time = end_gen_time - start_gen_time
            pob_time += elapsed_gen_time
            
            msg_info_generacion(gen, poblacion, elapsed_gen_time)
            historial_generacional[gen,0] = [poblacion,cruces]
            historial_generacional[gen,1] = elapsed_gen_time
            
        msg_tiempo_pob(i, pob_time)
        gen_results.append(historial_generacional)
    end_exp_time = time.time()
    elapsed_exp_time = end_exp_time - start_exp_time
    msg_info_exp(f"Tiempo de experimentación: {elapsed_exp_time}")
    
    return gen_results,elapsed_exp_time

# Experimento combinacion PMX, y Heuristic mutation

En este experimento a partir de la población incial, tomar en cuenta que hay 5 poblaciones inciales con las cuales debe realizarse el experimento, se obtienen nuevas generaciones de individuos y una vez alcanza el 80% de individuos aptos, esto quiere decir que los individuso deben superar más del 0.8 de aptitud, o se llega un 10 generaciones el algoritmo se detiene.

**Condiciones de termino:**

* Epsilon:
    * Cuando el 80% de la población alcanza una aptitud mayor de 0.8
* Generaciones:
    * Cuando se llega a la generación número 10

**Operadores:**
* Método de competencia genética para la creación de las poblaciones
* Operador de selección *Ruleta*
* Operador de cruce *PMX*
* Operador de mutación  *Hueristico*

## Evaluar y Ordenar la poblacion

La función de ordenar población ordena de mayor a menor.

Este problema es un problema para encontrar las rutas con menor costo

In [140]:
resultados1,tiempo_ejecucion1 = experimentacion(poblaciones_iniciales=pobs,max_gen=500,op_seleccion=0,op_cruza=3,mode="min",mutar=True)

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 68260.256915
mutar
...................................................................
Generación: 0
Costo promedio: 60241.13490000001
Varianza del costo en la genearción: 19312850.13043802
Desviación estándar del costo en la genearción: 4394.638794080581
Diversidad promedio: 16.866262626262625 genes
Menor costo de toda la generación : 49408.6723
Mayor costo de toda la generación : 66885.404
Tiempo de ejecución de la genaración 0: 7.460499048233032
...................................................................
mutar


KeyboardInterrupt: 

In [141]:
def show_reporte(resultados, msg):
    reportes_iniciales = np.array([get_report(i + 1, p) for i, p in enumerate(pobs)])
    # Comportamiento de las poblaciones iniciales
    fig = make_subplots(
        rows=6,
        cols=1,
        subplot_titles=["Costo promedio", "Varianza del costo", "Desviación estándar","Individuos con menor costo población inicial","Individuos mayor costo población inicial","Diversidad de genes"],
    )

    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 1],
            mode="markers+lines",
            name="Costo promedio inicial",
            marker=dict(symbol="star-diamond",size=8),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 2],
            mode="markers+lines",
            marker=dict(symbol="diamond-x",size=8),
            name="Varianza costo",
        ),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 3],
            mode="markers+lines",
            marker=dict(symbol="octagon",size=8),
            name="Desviación costo",
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:,0],
            y=reportes_iniciales[:,4],
            mode="markers+lines",
            marker=dict(symbol="bowtie",size=8),
            name="Costo menor"
        ),row=4,col=1
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:,0],
            y=reportes_iniciales[:,5],
            mode="markers+lines",
            marker=dict(symbol="bowtie",size=8),
            name="Costo mayor"
        ),row=5,col=1
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:,0],
            y=reportes_iniciales[:,6],
            mode="markers+lines",
            marker=dict(symbol="hexagram",size=8),
            name="Costo mayor"
        ),row=6,col=1
    )

    fig.update_layout(height=1000, width=900)  # Ajusta el tamaño de la figura
    # Agrega títulos a los ejes x y y de cada gráfica
    fig.update_layout(
        title = f"{msg}",
        xaxis1=dict(title="número población"),  # Eje x de la primera gráfica
        yaxis1=dict(title="Costo"),  # Eje y de la primera gráfica
        xaxis2=dict(title="número población"),  # Eje x de la segunda gráfica
        yaxis2=dict(title="Costo"),  # Eje y de la segunda gráfica
        xaxis3=dict(title="número población"),  # Eje x de la tercera gráfica
        yaxis3=dict(title="Costo"),  # Eje y de la tercera gráfica
        xaxis4=dict(title="número población"),  # Eje x de la tercera gráfica
        yaxis4=dict(title="Costo"),  # Eje y de la tercera gráfica
        xaxis5=dict(title="número población"),  # Eje x de la tercera gráfica
        yaxis6=dict(title="Costo"),  # Eje y de la tercera gráfica
    )
    fig.show()


show_reporte(resultados1, "Experimento número 1, sin implementar mutación")

NameError: name 'resultados1' is not defined

In [206]:
gen_reports = np.zeros((5,500,8))
for i,rg in enumerate(resultados1):
    rg_tiempos = rg[:,1]
    rg_res = rg[:,0]
    
    rg_res_pobs = np.array([respob[0] for respob in rg_res])
    rg_report = np.array([get_report(k+1,rpob,rg_tiempos[k]) for k,rpob in enumerate(rg_res_pobs)])
    gen_reports[i] = rg_report

    #rg_res_cros = np.array([rescros[1] for rescros in rg_res])
    
# print(tiempo_ejecucion1)
# for i,res in enumerate(resultados1):
#     n_pob = i
#     for j,gen in enumerate(res):
#         historico = gen[0]
#         tiempo_ejecucion = gen[1]
#         registr_mutante = None
#         if len(historico) == 3:
#             print("Mutante")
#         print(historico[0].shape)
#         print(historico[1].shape)
#     break

In [207]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming gen_reports has shape (5, 10, 8)

fig = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=[
        "Costo promedio",
        "Varianza",
        "Desviación estándar",
        "Costo menor",
        "Costo mayor",
        "Diversidad genética",
    ],
    
)

colors = [
    '#f5bde6',  # muted blue
    '#c6a0f6',  # safety orange
    '#ed8796',  # cooked asparagus green
    '#a6da95',  # brick red
    '#7dc4e4',  # muted purple
]

# Plot each metric for each population
for i, pop in enumerate(gen_reports):
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 1],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 2],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 3],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 4],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=2,
        col=2,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 5],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 6],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=3,
        col=2,
    )

# Customize axis labels
for i in range(6):
    fig.update_xaxes(title_text="Generación", row=(i // 2) + 1, col=(i % 2) + 1)

fig.update_layout(
    height=1200, width=1000, title="Comportamiento de las 5 poblaciones iniciales"
)
fig.show()

In [208]:
# for i,fila in enumerate(resultados[-1]):
#     msg_info_generacion(i,fila[0],fila[1])
    
# for i,pob_A in enumerate(pobs):
#     aux_res = resultados[i]
#     tiempo = np.sum(aux_res[:,1])
#     msg_info_generacion(i,pob_A,tiempo)

In [209]:
"""
Pruebas para la codificacion con ascii
# a = np.random.permutation(18)
a = np.arange(18)

aux = [individuo_toString(c.astype(int)) for c in a]
print(aux)

# for ciudad in a:
#     print(chr(ciudad+33))
    

aux = [chr(c+48)for c in a]

print(aux)

numeros = [ord(c) - 48 for c in aux]
print(numeros)
    
"""

'\nPruebas para la codificacion con ascii\n# a = np.random.permutation(18)\na = np.arange(18)\n\naux = [individuo_toString(c.astype(int)) for c in a]\nprint(aux)\n\n# for ciudad in a:\n#     print(chr(ciudad+33))\n    \n\naux = [chr(c+48)for c in a]\n\nprint(aux)\n\nnumeros = [ord(c) - 48 for c in aux]\nprint(numeros)\n    \n'

In [210]:
a = np.zeros((1,3))
b = np.zeros((2,5))
c = np.ones((4,4))

fila = [a,b,c]
print(fila)

[array([[0., 0., 0.]]), array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]]), array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])]


## Experimento 2

**Condiciones de termino:**

* Epsilon:
    * Cuando el 80% de la población alcanza una aptitud mayor de 0.8
* Generaciones:
    * Cuando se llega a la generación número 10

**Operadores:**
* Método de competencia genética para la creación de las poblaciones
* Operador de selección *Ruleta*
* Operador de cruce *CX*
* Operador de mutación  *Scramble*

In [142]:
resultados2,tiempo_ejecucion2 = experimentacion(poblaciones_iniciales=pobs,max_gen=100,op_seleccion=0,op_cruza=4,mode="min",mutar=True)

Inicio del algoritimo genético
Población incial #1
Costo promedio inicial 68260.256915
mutar
...................................................................
Generación: 0
Costo promedio: 60320.48099900001
Varianza del costo en la genearción: 31215292.250105835
Desviación estándar del costo en la genearción: 5587.064725784537
Diversidad promedio: 16.8379797979798 genes
Menor costo de toda la generación : 39829.0797
Mayor costo de toda la generación : 67899.7513
Tiempo de ejecución de la genaración 0: 5.602283716201782
...................................................................
mutar
...................................................................
Generación: 1
Costo promedio: 55657.86039999999
Varianza del costo en la genearción: 18293504.777130116
Desviación estándar del costo en la genearción: 4277.090690776865
Diversidad promedio: 16.703434343434342 genes
Menor costo de toda la generación : 39829.0797
Mayor costo de toda la generación : 61038.582
Tiempo de ejecución de

In [144]:
def show_reporte(resultados, msg):
    reportes_iniciales = np.array([get_report(i + 1, p) for i, p in enumerate(pobs)])
    # Comportamiento de las poblaciones iniciales
    fig = make_subplots(
        rows=6,
        cols=1,
        subplot_titles=["Costo promedio", "Varianza del costo", "Desviación estándar","Individuos con menor costo población inicial","Individuos mayor costo población inicial","Diversidad de genes"],
    )

    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 1],
            mode="markers+lines",
            name="Costo promedio inicial",
            marker=dict(symbol="star-diamond",size=8),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 2],
            mode="markers+lines",
            marker=dict(symbol="diamond-x",size=8),
            name="Varianza costo",
        ),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:, 0],
            y=reportes_iniciales[:, 3],
            mode="markers+lines",
            marker=dict(symbol="octagon",size=8),
            name="Desviación costo",
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:,0],
            y=reportes_iniciales[:,4],
            mode="markers+lines",
            marker=dict(symbol="bowtie",size=8),
            name="Costo menor"
        ),row=4,col=1
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:,0],
            y=reportes_iniciales[:,5],
            mode="markers+lines",
            marker=dict(symbol="bowtie",size=8),
            name="Costo mayor"
        ),row=5,col=1
    )
    fig.add_trace(
        go.Scatter(
            x=reportes_iniciales[:,0],
            y=reportes_iniciales[:,6],
            mode="markers+lines",
            marker=dict(symbol="hexagram",size=8),
            name="Costo mayor"
        ),row=6,col=1
    )

    fig.update_layout(height=1000, width=900)  # Ajusta el tamaño de la figura
    # Agrega títulos a los ejes x y y de cada gráfica
    fig.update_layout(
        title = f"{msg}",
        xaxis1=dict(title="número población"),  # Eje x de la primera gráfica
        yaxis1=dict(title="Costo"),  # Eje y de la primera gráfica
        xaxis2=dict(title="número población"),  # Eje x de la segunda gráfica
        yaxis2=dict(title="Costo"),  # Eje y de la segunda gráfica
        xaxis3=dict(title="número población"),  # Eje x de la tercera gráfica
        yaxis3=dict(title="Costo"),  # Eje y de la tercera gráfica
        xaxis4=dict(title="número población"),  # Eje x de la tercera gráfica
        yaxis4=dict(title="Costo"),  # Eje y de la tercera gráfica
        xaxis5=dict(title="número población"),  # Eje x de la tercera gráfica
        yaxis6=dict(title="Costo"),  # Eje y de la tercera gráfica
    )
    fig.show()


show_reporte(resultados2, "Experimento número 2, sin implementar mutación")

In [148]:
gen_reports = np.zeros((5,100,8))
for i,rg in enumerate(resultados2):
    rg_tiempos = rg[:,1]
    rg_res = rg[:,0]
    
    rg_res_pobs = np.array([respob[0] for respob in rg_res])
    rg_report = np.array([get_report(k+1,rpob,rg_tiempos[k]) for k,rpob in enumerate(rg_res_pobs)])
    gen_reports[i] = rg_report

    #rg_res_cros = np.array([rescros[1] for rescros in rg_res])
    
# print(tiempo_ejecucion1)
# for i,res in enumerate(resultados1):
#     n_pob = i
#     for j,gen in enumerate(res):
#         historico = gen[0]
#         tiempo_ejecucion = gen[1]
#         registr_mutante = None
#         if len(historico) == 3:
#             print("Mutante")
#         print(historico[0].shape)
#         print(historico[1].shape)
#     break

In [149]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming gen_reports has shape (5, 10, 8)

fig = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=[
        "Costo promedio",
        "Varianza",
        "Desviación estándar",
        "Costo menor",
        "Costo mayor",
        "Diversidad genética",
    ],
    
)

colors = [
    '#f5bde6',  # muted blue
    '#c6a0f6',  # safety orange
    '#ed8796',  # cooked asparagus green
    '#a6da95',  # brick red
    '#7dc4e4',  # muted purple
]

# Plot each metric for each population
for i, pop in enumerate(gen_reports):
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 1],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 2],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 3],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 4],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=2,
        col=2,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 5],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=pop[:, 0],
            y=pop[:, 6],
            name=f"Población {i+1}",
            mode="markers+lines",
            marker=dict(symbol="star-dot", size=10,color=colors[i]),
            legendgroup=f"Población {i+1}"
        ),
        row=3,
        col=2,
    )

# Customize axis labels
for i in range(6):
    fig.update_xaxes(title_text="Generación", row=(i // 2) + 1, col=(i % 2) + 1)

fig.update_layout(
    height=1200, width=1000, title="Comportamiento de las 5 poblaciones iniciales"
)
fig.show()